# Hands On Tutorial 3: Multitask Deep Learning for Drug Discovery

Implemntation of the model described in https://arxiv.org/pdf/1502.02072.pdf

### Imports and parameters

In [1]:
import keras
import keras.backend as K
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD

import pandas as pd
import numpy as np
import zipfile
import requests
import shutil
import os

from helpers.extra_metrics import explained_variance_score, r2_score

Using TensorFlow backend.


In [2]:
""" 
add imports for layers and loss functions here
"""
from keras.layers import Input, Dense, Conv1D, LSTM, Dropout, Flatten
from keras.losses import MSE

In [85]:
MAX_LEN = 120
VAL_SIZE = 20000
LEARNING_RATE = .000001
EPOCHS = 50
BATCH_SIZE = 64

### Dataset preparation

We will use the [QM9 dataset](http://quantum-machine.org/datasets/):
* L. Ruddigkeit, R. van Deursen, L. C. Blum, J.-L. Reymond, Enumeration of 166 billion organic small molecules in the chemical universe database GDB-17, J. Chem. Inf. Model. 52, 2864–2875, 2012.
* R. Ramakrishnan, P. O. Dral, M. Rupp, O. A. von Lilienfeld, Quantum chemistry structures and properties of 134 kilo molecules, Scientific Data 1, 140022, 2014. [bibtex]

The QM9 dataset contains 133885 organic molecules, represented as SMILES strings, with 15 properties each. We will train a regression model to predict all 15 properties in a multitask settings.

In [ ]:
# download dataset
url = 'https://s3.us-east-2.amazonaws.com/weizmann-dl-workshop/data_qm9.pkl.zip'
file_path = 'datasets/qm9.pkl.zip'

In [ ]:
os.makedirs('datasets/', exist_ok=True)
r = requests.get(url, auth=('usrname', 'password'), verify=False,stream=True)
r.raw.decode_content = True
with open(file_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [ ]:
""" ONLY IF THE PREVIOUS FAILED TO DOWNLOAD """
!mkdir -p datasets
!wget --no-check-certificate $url -O $file_path

In [ ]:
# extract dataset
with zipfile.ZipFile(file_path, 'r') as file:
    file.extractall('datasets/')

In [4]:
# preprocess data
data = pd.read_pickle('datasets/data_qm9.pkl')
labels = ['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

properties = {}
for i, key in enumerate(labels):
    properties[key] = data['properties'][:, i]
    
properties['smiles'] = data['smiles_optimized']

df = pd.DataFrame(properties)
df = df[['smiles'] + labels]
df.head(20)

,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv
0,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469
1,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316
2,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002
3,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574
4,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278
5,C=O,285.48839,38.982300,34.298920,2.1089,14.18,-0.2670,-0.0406,0.2263,59.9891,0.026603,-114.483613,-114.480746,-114.479802,-114.505268,6.413
6,CC,80.46225,19.906490,19.906330,0.0000,23.95,-0.3385,0.1041,0.4426,109.5031,0.074542,-79.764152,-79.760666,-79.759722,-79.787269,10.098
7,CO,127.83497,24.858720,23.978720,1.5258,16.97,-0.2653,0.0784,0.3437,83.7940,0.051208,-115.679136,-115.675816,-115.674872,-115.701876,8.751
8,CC#C,160.28041,8.593230,8.593210,0.7156,28.78,-0.2609,0.0613,0.3222,177.1963,0.055410,-116.609549,-116.605550,-116.604606,-116.633775,12.482
9,CC#N,159.03567,9.223270,9.223240,3.8266,24.45,-0.3264,0.0376,0.3640,160.7223,0.045286,-132.718150,-132.714563,-132.713619,-132.742149,10.287


In [5]:
# set maximum length
MAX_LEN = df.smiles.str.len().max()

#### Task 1: convert smiles strings into one-hot representation and add to the dataframe in a column named 'one_hot' 
  
hint: use zero padding to make the length equal to MAX_LEN

In [6]:
"""
Convert SMILES to one-hot here
"""

smiles = df.smiles.tolist()
characters = set()

for s in smiles:
    for char in s:
        if char not in characters:
            characters.add(char)

characters = sorted(list(characters))
num_chars = len(characters)

print('Number of strings:', len(smiles))
print('Number of unique chars:', num_chars)

chars_index = dict([(char, i) for i, char in enumerate(characters)])

one_hot = np.zeros((len(df), MAX_LEN, num_chars), dtype='float32')

for i, s in enumerate(smiles):
    for j, char in enumerate(s):
        one_hot[i, j, chars_index[char]] = 1.
        
df['one_hot'] = one_hot.tolist()
df.head()

Number of strings: 133885
Number of unique chars: 21


,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,one_hot
0,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [7]:
# split to train/validation sets
x_train = np.array([np.array(oh) for oh in df.one_hot.values[:-VAL_SIZE]])
x_val = np.array([np.array(oh) for oh in df.one_hot.values[-VAL_SIZE:]])

y_train = df[labels].values[:-VAL_SIZE]
y_val = df[labels].values[-VAL_SIZE:]

### Building Model Architecture

#### Task 2: define model architecture

In [8]:
inp = Input(shape=(MAX_LEN, num_chars))

In [86]:
"""
Build your model here.
Eventually the variable 'outputs' should be a list of all the model outputs, one for each task.
"""
flat = Flatten()(inp)

dense1 = Dense(units=200, activation='relu')(flat)
dense2 = Dense(units=150, activation='relu')(dense1)
dense3 = Dense(units=100, activation='relu')(dense2)
dense4 = Dense(units=50, activation='relu')(dense3)
dense5 = Dense(units=30, activation='relu')(dense4)

In [87]:
output = Dense(units=15, 
               activation='linear', 
               kernel_regularizer=None)(dense5)

In [88]:
model = Model(inputs=inp, outputs=output)

### Compile model and set loss function and metrics


#### Task 3: define loss function

In [89]:
"""
Define your loss function here
"""
loss = MSE

In [90]:
# define metrics
metrics = [MSE, explained_variance_score, r2_score]

# compile model
model.compile(loss=loss,
              metrics=metrics,
              optimizer=SGD(lr=LEARNING_RATE, nesterov=True))

### Model Training


In [91]:
# set callbacks
callbacks = [ReduceLROnPlateau(factor=.2, patience=4, verbose=1),
             EarlyStopping(monitor='val_loss', patience=10)]

In [92]:
# train model
model.fit(x=x_train,
          y=y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          callbacks=callbacks,
          validation_data=(x_val, y_val))

Train on 113885 samples, validate on 20000 samples
Epoch 1/50
113885/113885 [==============================] - 13s 118us/step - loss: 394995.4050 - mean_squared_error: 394995.4050 - explained_variance_score: -0.1688 - r2_score: -0.0039 - val_loss: 186329.7742 - val_mean_squared_error: 186329.7742 - val_explained_variance_score: -0.3057 - val_r2_score: -0.0048
Epoch 2/50
113885/113885 [==============================] - 12s 108us/step - loss: 394858.5012 - mean_squared_error: 394858.5012 - explained_variance_score: -0.2905 - r2_score: -0.0029 - val_loss: 186088.5254 - val_mean_squared_error: 186088.5254 - val_explained_variance_score: -0.6382 - val_r2_score: -0.0034
Epoch 3/50
113885/113885 [==============================] - 12s 105us/step - loss: 360466.2759 - mean_squared_error: 360466.2759 - explained_variance_score: -533.8281 - r2_score: 0.2464 - val_loss: 10388.2704 - val_mean_squared_error: 10388.2704 - val_explained_variance_score: -2.2360 - val_r2_score: 0.9554
Epoch 4/50
113885/

Epoch 28/50
113885/113885 [==============================] - 12s 107us/step - loss: 258389.9124 - mean_squared_error: 258389.9124 - explained_variance_score: -2.0594 - r2_score: 0.9860 - val_loss: 4599.7539 - val_mean_squared_error: 4599.7539 - val_explained_variance_score: -1.6552 - val_r2_score: 0.9806
Epoch 29/50
113885/113885 [==============================] - 12s 105us/step - loss: 258357.0599 - mean_squared_error: 258357.0599 - explained_variance_score: -2.1737 - r2_score: 0.9863 - val_loss: 4612.6602 - val_mean_squared_error: 4612.6602 - val_explained_variance_score: -1.6703 - val_r2_score: 0.9805
Epoch 30/50
113885/113885 [==============================] - 12s 105us/step - loss: 258340.5019 - mean_squared_error: 258340.5019 - explained_variance_score: -2.1408 - r2_score: 0.9864 - val_loss: 4595.2929 - val_mean_squared_error: 4595.2929 - val_explained_variance_score: -1.6522 - val_r2_score: 0.9806
Epoch 31/50
113885/113885 [==============================] - 12s 104us/step - loss

### Model Evaluation


In [93]:
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score

In [94]:
y_pred = model.predict(x=x_val)

In [95]:
print('##### Model Evaluation #####')
print('Mean Squared Error: %f' % mean_squared_error(y_true=y_val, y_pred=y_pred))
print('Explained Variance: %f' % explained_variance_score(y_true=y_val, y_pred=y_pred))
print('R2 Score: %f' % r2_score(y_true=y_val, y_pred=y_pred))

##### Model Evaluation #####
Mean Squared Error: 4382.275416
Explained Variance: -2.325896
R2 Score: 0.449443


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)
